<font size="6">Машинный перевод</font>

# Задача машинного перевода

Ранее мы рассматривали в основном задачу классификации: каждому тексту сопоставляется метка класса из заданного списка.

Другая важная задача обработки естественного языка — машинный перевод: последовательности на одном языке сопоставляется последовательность на другом языке, передающая тот же смысл.

Рассмотрим задачу перевода с английского на французский язык:
- Исходное предложение: I am a student.
- Целевое предложение: Je suis étudiant.

<center><img src ="https://i.postimg.cc/bwXzwx5c/machine-translation.jpg" width="600"></center>

Что нужно для перевода текста?
1. Понимать отдельные слова
2. Понимать взаимодействие между словами (синтаксис)
3. Переводить слова (с учетом контекста)
4. Составлять осмысленный и связный текст

Для перевода текста можно разбить исходное предложение на несколько фрагментов, затем переводить его по фразам.

Однако у такого подхода есть проблемы:
- одному слову в исходном предложении (*forced*) может соответствовать несколько слов в целевом предложении (*a forcé*)
- порядок слов может меняться (*exceptional measures* vs. *des mesures exceptionnelles*)

Следовательно, возникают сложности с построением выравнивания слов (word alignment).

<center><img src ="https://i.postimg.cc/SNK4dyXs/word-alignment.png" width="300"></center>

Другая возможность — прочитать все исходное предложение целиком, проанализировать его значение, а затем произвести перевод.

В этом случае модель состоит из двух блоков:
- кодировщик: строит векторное представление для исходного предложения (кодирует);
- декодировщик: генерирует целевое предложение на основе этого векторного представления (раскодирует).

<center><img src="https://i.ibb.co/h2srmnP/encoder-decoder.png" width="800"></center>

В качестве блоков кодировщика и декодировщика могут использоваться нейронные сети с различной архитектурой.
1. Изначально это были рекуррентные нейронные сети, которые хорошо подходят для обработки последовательных данных;
2. Позже была предложена архитектура Трансформер, которая лежит в основе современных языковых моделей.

В сегодняшней лекции мы рассмотрим обе эти архитектуры и их применение в задаче машинного перевода.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/rnn_transformer.png" width="700"></center>

<center><em>Source: <a href="https://shchegrikovich.substack.com/p/rnn-vs-transformers-or-how-scalability">RNN vs Transformers</a></em></center>

# Рекуррентные нейронные сети

**Рекуррентные нейронные сети (recurrent neural networks, RNN)** — это класс нейронных сетей, которые применяются для обработки последовательных данных.

Они применяются в широком перечне задач: от **распознавания речи** до **генерации подписей** к изображениям.

Эти задачи объединяет необходимость работы с контекстом и извлечения информации, сформированной при обработке одной части данных, для обработки других частей.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/sequence_data.png" width="700"></center>

<center><em>Source: <a href="https://cbare.github.io/2019-01-27/deep-learning-sequence-models.html">Deep Learning - Sequence Models</a></em></center>

### Рекуррентная ячейка и рекуррентный слой

Рассмотрим работу рекуррентных сетей применительно к текстовым данным.

Каждое слово $x_t$ в предложении последовательно обрабатывается с помощью **ячейки RNN**.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/RNN_cell.png" width="700"></center>

<center><em>Source: <a href="http://vbystricky.ru/2021/05/rnn_lstm_gru_etc.html">RNN, LSTM, GRU</a></em></center>

1. На вход ячейки поступает входной вектор $x_t$ —  эмбеддинг слова с индексом $t$. Он имеет фиксированный размер $k$.

2. Ячейка принимает еще один параметр $h_{t-1}$ — **скрытое состояние** или **память** (hidden state). Это вектор, хранящий информацию о предшествующем контексте. Он тоже имеет фиксированный размер $n$.

3. Вектор $x_t$ умножается на матрицу $W^{nk}$ (размер $n \times k$), которая содержит обучаемые веса: $W^{nk} \cdot x_t$. Получаем новый вектор размера $n$.

4. Вектор $h_{t-1}$ умножается на другую матрицу весов $W^{nn}$ (размер $n \times n$): $ W^{nn} \cdot h_{t-1} $. Получаем новый вектор размера $n$.

5. Получившиеся векторы имеют одинаковый размер, их можно сложить: $W^{nk} \cdot x_t + W^{nn} \cdot h_{t-1}$.

6. К получившемуся вектору применим функцию активации — гиперболический тангенс: $tanh(W^{nk} \cdot x_t + W^{nn} \cdot h_{t-1})$. Это и будет новым скрытым состоянием $h_t$. Оно зависит от предыдущего состояния $h_{t-1}$ и текущего элемента последовательности $x_t$.

7. Рассчитанное скрытое состояние $h_t$ является представлением текущего слова $x_t$ с учетом предшествующего контекста и передается в качестве старого скрытого состояния для следующего слова $x_{t+1}$.

Для обработки всей последовательности слов используется **слой RNN**, где ячейка применяется ко всем словам в цикле.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/RNN_layer.png" width="700"></center>

<center><em>Source: <a href="http://vbystricky.ru/2021/05/rnn_lstm_gru_etc.html">RNN, LSTM, GRU</a></em></center>

Когда первый токен $x_0$ подается в ячейку, вектор памяти $h$ инициализируется нулями.

Вектор памяти $h_0$ для первого токена $x_0$ передается в следующую ячейку, обрабатывающую второй токен $x_1$. Вектор $h_0$ также является контекстуализированным представлением токена $x_0$ — $y_0$.

Для каждого последующего токена вектор памяти учитывает контекст из предыдущих токенов.

Вектор $y_t$ является не только контекстуализированным представлением для последнего токена $x_t$, но и вектором всего предложения, т.к. содержит информацию обо всех токенах.

В зависимости от типа задачи, можно использовать контекстуализированные вектора для каждого слова или только вектор всего предложения (последнего слова).

## RNN слой в PyTorch

В PyTorch есть слой — `torch.nn.RNN` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html), который реализует логику, описанную выше.

Также есть сущность `torch.nn.RNNCell` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html), которая реализует вычисления на одном такте времени.

Слой `nn.RNN` фактически является оберткой, которая вызывает `nn.RNNCell` в цикле по длине входной последовательности.

Параметры слоя `nn.RNN`:

* **`input_size`** — размерность $\large x_t$, целое число.

* **`hidden_size`** — размерность $\large h_t$, целое число. Фактически это количество нейронов в рекуррентном слое.


Техническая особенность рекуррентных слоев в PyTorch: по умолчанию ожидаемые размерности входа такие:

**`[длина последовательности, размер батча, размерность входа]`**

Однако если при создании слоя указать `batch_first=True`, то можно подавать входные значения в более привычном формате, когда размер батча стоит на первом месте:

**`[размер батча, длина последовательности, размерность входа]`**



In [ ]:
import torch

torch.manual_seed(42)

rnn = torch.nn.RNN(input_size=3, hidden_size=2, batch_first=True)

dummy_batched_seq = torch.randn((16, 57, 3))  # batch_size, seq_len, input_size
out, h = rnn(dummy_batched_seq)

print("Input shape:".ljust(20), f"{dummy_batched_seq.shape}")
print("Out shape:".ljust(20), f"{out.shape}")
print("Last hidden state shape:".ljust(20), f"{h.shape}")

При вызове слой возвращает два объекта:
* `out` — последовательность скрытых состояний,
* `h` — скрытое состояние на последнем такте.

Мы указали `batch_first=True`, при этом `out` сохранил последовательность размерностей, как у входа, а вот у `h` размерность батча встала на второе место.

In [ ]:
h_batch_first = h.permute(1, 0, 2)

print(f"h is last out: {(h_batch_first == out[:, -1:, :]).all().item()}")

### Многослойные RNN

RNN-блоки можно объединять в слои, накладывая их друг на друга. Для этой операции в `torch.nn.RNN` есть аргумент `num_layers`, с помощью которого можно указать количество слоёв.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/rnn_multiple_layers.png" width="500"></center>

In [ ]:
dummy_input = torch.randn((16, 6, 3))  # batch_size, seq_len, input_size
rnn = torch.nn.RNN(input_size=3, hidden_size=2, num_layers=2, batch_first=True)

out, h = rnn(dummy_input)

print()
print("Out:\n", out.shape)  # Hidden states for all elements from top layer
print("h:\n", h.shape)  # Hidden states for last element for all layers

## Проблемы RNN

Теоретически, можно было бы сразу пропустить все данные через сеть и затем вычислить градиент, однако возникнут следующие проблемы:

 - большие последовательности не поместятся в памяти,
 - так как цепочка будет очень длинной, возникнет затухание/взрыв градиента,
 - по мере прохождения сигнала по цепи контекст затирается.


Допустим, у нас есть длинная последовательность. Если мы сразу предсказываем, то в каждый момент времени нужно распространить Loss. И все ячейки нужно обновить во время backpropogation. Все градиенты нужно посчитать. Возникают проблемы, связанные с нехваткой памяти.

Есть специальные тесты для проверки, контекст какой длины использует RNN при предсказании. Если мы делаем предсказание только в последней ячейке, может оказаться, что используется, скажем, информация только о последних 10 словах предложения.

Функция активации Tanh постепенно затирает контекст.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/backprop_through_time.png" width="700"><center>

<center><em>Source: <a href="http://cs231n.stanford.edu/slides/2021/lecture_10.pdf">CS231n: Recurrent Neural Network</a></em></center>

Затухающий/взрывающийся градиент (Vanishing/exploding gradient) — явления затухающего и взрывающегося градиента часто встречаются в контексте RNN. И при большой длине последовательности это становится критичным. Причина в том, что зависимость величины градиента от числа слоёв экспоненциальная, поскольку веса умножаются многократно.

$dL ∝ (W)^N:$

$W > 1 \rightarrow$ взрыв, $W < 1 \rightarrow$ затухание.

<img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/simple_rnn_backprop.png" width="1000">

Один из путей решения проблемы — **градиентное отсечение** (Gradient truncating) — метод, который ограничивает максимально допустимое значение градиента, позволяя избежать градиентного взрыва.

А от затухания градиента может помочь **пропускание** **градиента по частям**, на сколько-то шагов по времени назад или вперёд, а не через всю нейросеть. Да, градиент будет не совсем точно считаться, и мы будем терять в качестве. Но это экономит память.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/truncated_backprop.png"  width="700"></center>

<center><em>Source: <a href="http://cs231n.stanford.edu/slides/2021/lecture_10.pdf">CS231n: Recurrent Neural Network</a></em></center>

## LSTM (Long Short-Term Memory)

Обычная RNN имела множество проблем, в том числе в ней очень быстро затухала информация о предыдущих элементах последовательности. Помимо этого были проблемы с затуханием/взрывом градиента.

Эти проблемы были частично решены в LSTM, предложенной в [Long Short-Term Memory (Hochreiter & Schmidhuber, 1997) 🎓[article]](http://www.bioinf.jku.at/publications/older/2604.pdf).

В обычной RNN-ячейке был только один путь передачи информации. На каждом шаге мы конкатенировали предыдущее скрытое состояние с текущим входом и пропускали их через линейный слой и активацией:

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >

<tr>
<td>

<center><img src = "https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/simple_rnn_h_state.png" width="500"></center>

</td>
<td>

<table >
<tr>
<td>

$\large h_t = \tanh(W \cdot [h_{t-1}, x_t])$

</td>

</tr>


</table>

</td>
</tr>

</table>








При этом информация о предыдущих элементах последовательности очень быстро затухает и теряется общая информация о предложении.

Структура ячейки LSTM намного сложнее. Здесь есть целых 4 линейных слоя, каждый из которых выполняет разные задачи.

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >
<tr>
<td>

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_chain.png" width="500"></center>
<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_chain_notation.png" width="700"></center>

</td>
<td>

<table >
<tr>
<td>

$\large f_t = σ(W_f \cdot [h_{t-1}, x_t])\ \ \ \ $

</td>
<td>

$$\large \text{forget  gate}$$

</td>
</tr>

<tr>
<td>

$\large i_t = σ(W_i \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{input gate}$$

</td>
</tr>

<tr>
<td>

$\large o_t = σ(W_o \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{output gate}$$

</td>
</tr>

<tr>
<td>

$\large c^\prime_t = \tanh(W_c \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{candidate cell state}$$

</td>
</tr>

<tr>
<td>

$\large c_t = f_t\otimes c_{t-1} + i_t \otimes c^\prime_t$

</td>
<td>

$$\large \text{cell state}$$

</td>
</tr>

<tr>
<td>

$\large h_t = o_t\otimes \tanh(c_t)$

</td>
<td>

$$\large  \text{hidden state}$$

</td>
</tr>
</table>

</td>
</tr>

</table>








Главное нововведение: в LSTM добавлен путь $c$, который по задумке должен этот общий контекст сохранять.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_c_state_highway.png" width="500"></center>

Другими словами, путь $c$ (cell state, иногда называется highway, магистраль)  помогает нейросети сохранять важную информацию, встретившуюся в какой-то момент в последовательности в прошлом, все время, пока эта информация требуется.

По формулам также видно, как возросла сложность.

Отличие от RNN состоит в том, что кроме $h$ возвращается еще и $c$.

In [ ]:
import torch.nn as nn


lstm = nn.LSTM(input_size=3, hidden_size=2, batch_first=True)
input = torch.randn(16, 57, 3)  # batch_size, seq_len, input_size
out, (h, c) = lstm(input)  # h and c returned in tuple

print("Input shape:".ljust(15), input.shape)
print("Shape of h".ljust(15), h.shape)  # 1, batch_size, hidden_size
print("Shape of c".ljust(15), c.shape)  # 1, batch_size, hidden_size
print("Output shape:".ljust(15), out.shape)  # batch_size, seq_len, hidden_size

## GRU (Gated Recurrent Unit)

Самая известная модификация LSTM — GRU. Она более компактна за счет сильных упрощений в сравнении со стандартной LSTM.

Главные изменения: объединены forget и input gates, слиты $h_t$ и $c_t$, которые в обычной LSTM только участвовали в формировании друг друга.

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >
<tr>
<td>

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/gru_basic_block.png" width="500"></center>

</td>
<td>

<table >
<tr>
<td>

$\large z_t = \sigma(W_z \cdot [h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large r_t = \sigma(W_r \cdot [h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large \tilde h_t = \tanh(W \cdot [r_t \otimes h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large h_t = (1-z_t) \otimes h_{t-1} + z_t \otimes \tilde h_t$

</td>


</table>








</td>
</tr>

</table>








In [ ]:
gru = nn.GRU(input_size=3, hidden_size=2, batch_first=True)
input = torch.randn(16, 57, 3)  # batch_size, seq_len, input_size
out, h = gru(input)

print("Input shape:".ljust(15), input.shape)
print("Shape of h:".ljust(15), h.shape)  # 1, batch_size, hidden_size
print("Output shape:".ljust(15), out.shape)  # batch_size, seq_len, hidden_size

Практический опыт исследователей: иногда лучше работает GRU, иногда — LSTM. Точный рецепт успеха сказать нельзя.

## Типы задач

- Один к одному: на вход подается один элемент, на выходе получаем вероятность класса — классификация изображений.

- Один ко многим: на вход приходит один элемент, а на выходе получаем целую последовательность — генерация текста по слову, текстовой подписи по изображению.

- Многие к одному: на вход сети подается последовательность, а в качестве выхода получаем один вектор вероятностей для классов — классификация текстов (анализ тональности, определение языка).

- Многие ко многим: преобразовать последовательность в последовательность
  - количество выходов равно количеству входов — тегирование (именованные сущности, части речи).
  - количество выходов сети не обязательно равно количеству входов — машинный перевод, суммаризация.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/RNN_tasks.png" width="800"></center>

<center><em>Source: <a href="https://dotnettutorials.net/lesson/recurrent-neural-network/">Recurrent Neural Network (RNNs)</a></em></center>

## RNN для машинного перевода

Машинный перевод относится к задаче преобразования одной последовательности в другую, длина которых может быть любой и не обязательно должна совпадать: "многие ко многим" или sequence-to-sequence (сокращенно seq2seq).

Модель для машинного перевода состоит из блока <font color="blue"> кодировщика </font> и <font color="red"> декодировщика</font>. Ниже представлена более подробная схема.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq.png" width="600"></center>

<center><em>Source: <a href="https://blog.paperspace.com/introduction-to-neural-machine-translation-with-bahdanaus-attention/">Introduction to Neural Machine Translation</a></em></center>

<div align="center">
    <table >
     <tr>
       <td>

Архитектура <font color="blue">кодировщика</font>:

- на вход первой ячейки RNN поступает вектор слова "I"<br>
и нулевое скрытое состояние,
- вектор слова "I" обрабатывается внутри первой ячейки RNN;
- на вход второй ячейки поступает вектор слова "am"<br>и измененный вектор слова "I" как скрытое состояние;
- вектор слова "am" обрабатывается внутри второй ячейки RNN;
- $\cdots$
- вектор слова "student" обрабатывается внутри четвертой ячейки RNN;
- измененный вектор слова "student" (= вектор предложения)<br>поступает на вход декодера как скрытое состояние.

</td>

<td>

Архитектура <font color="red">декодировщика</font>:
- на вход первой ячейки RNN поступает<br>вектор спецсимвола начала предложения <br>и вектор исходного предложения из энкодера как скрытое состояние;
-  вектор спецсимвола начала предложения<br>обрабатывается внутри первой ячейки RNN;
- на вход второй ячейки поступает вектор слова "Je"<br>и измененный вектор спецсимвола начала как скрытое состояние;
- вектор слова "Je" обрабатывается внутри второй ячейки RNN;
- $\cdots$
- вектор слова "étudiant" обрабатывается внутри четвертой ячейки RNN;
- измененные векторы каждого слова<br>передаются на линейный слой, применяется softmax;
- на выходе получаем 4 вектора, длина которых равна длине словаря, —<br>это распределение вероятностей для следующего элемента<br>при условии текущей последовательности.


</td>
     </tr>
    </table>
    </div>

Итак,
- <font color="blue">кодировщик</font> используется для получения конекстуализированного вектора исходного предложения;
- <font color="red">декодировщик</font> используется для генерации целевого предложения;
- вектор исходного предложения используется в качестве скрытого состояния на первом шаге генерации.

**Проблема Sequence-to-Sequence**: использование только конечного скрытого состояния энкодера для представления всей входной последовательности приводит к потере информации, особенно с начала последовательности;
- по мере того, как энкодер обрабатывает входную последовательность, ожидается, что в конечном скрытом состоянии будет собрана вся необходимая информация;
- когда последовательность становится длиннее, этому единственному состоянию труднее сохранять всю необходимую информацию из предыдущих частей последовательности.

Чтобы модель могла фокусироваться на различных частях входной последовательности и сохранять больше информации на протяжении всего процесса кодирования и декодирования, был предложен механизм внимания (attention mechanism).

[[paper] 🎓 Bahdanau D., Cho K., Bengio Y. (2014). Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_attention.png" width="600"></center>

<center><em>Source: <a href="https://blog.paperspace.com/introduction-to-neural-machine-translation-with-bahdanaus-attention/">Introduction to Neural Machine Translation</a></em></center>

Архитектура энкодера не меняется, все изменения касаются архитектуры декодера.
- на вход первой ячейки RNN также поступает вектор спецсимвола начала предложения и вектор исходного предложения из энкодера как скрытое состояние;
-  вектор спецсимвола начала предложения обрабатывается внутри первой ячейки RNN;
- для вектора спецсимвола начала предложения и векторов каждого слова исходной последовательности считается мера сходства — это и есть веса внимания (attention weights);
- если векторы некоторого слова целевого и исходного предложения схожи, то при переводе (генерации) данного слова декодер больше "обращает внимания" на него;
- вектор каждого слова исходного предложения умножается на свой вес внимания, затем все векторы складываются, получаем контекстный вектор (context vector);
- вектор спецсимвола начала предложения и контекстный вектор конкатенируются;
- конкатенированный вектор передается на линейный слой, применяется softmax;
- получаем вектор, длина которого равна длине словаря, — это распределение вероятностей для следующего элемента при условии спецсимвола начала предложения;
- на вход второй ячейки поступает вектор слова "Je" и измененный вектор спецсимвола начала как скрытое состояние;
- вектор слова "Je" обрабатывается внутри второй ячейки RNN;
- считаются веса внимания для слова "Je" и всех слов целевого предложения;
- $\cdots$
- для каждого слова получаем распределение вероятностей.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_equation.jpg" width="700"></center>

<center><em>Source: <a href="https://www.google.com/url?sa=i&url=https%3A%2F%2Fai.plainenglish.io%2Fintroduction-to-attention-mechanism-bahdanau-and-luong-attention-e2efd6ce22da&psig=AOvVaw1NlMR6N0XSxl6zYQwhAlYw&ust=1723824501708000&source=images&cd=vfe&opi=89978449&ved=0CAUQtaYDahcKEwiAhL3usPeHAxUAAAAAHQAAAAAQDw">Introduction to Attention Mechanism</a></em></center>

Механизм внимания позволил исправить некоторые проблемы применения рекуррентных сети, однако ряд недостатков остается:
- не способны запоминать дистантные зависимости
- возникает взрыв и затухание градиента
- вычисления проводятся последовательно и занимают много времени

Что если убрать рекуррентность и оставить только механизм внимания?

# Архитектура Трансформер

Архитектура, в которой отсутствует рекуррентность и используется только механизм внимания, получила название Трансформер.

[[paper] 🎓 Vaswani A. et al. (2017).Attention Is All You Need](https://arxiv.org/abs/1706.03762)

Как и в случае с рекуррентной сетью seq2seq, Трансформер состоит из блока энкодера и блока декодера.
- Энкодер обрабатывает исходную последовательность и кодирует ее.
- Декодер обрабатывает целевую последовательность с учетом информации из энкодера. Выход из декодера является предсказанием модели.

Блок энкодера состоит из 6 энкодеров, расположенных друг за другом. Блок декодера – это стек декодеров, представленных в том же количестве.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/transformer.jpg" width="860"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Все энкодеры идентичны по структуре, хотя и имеют разные веса. Каждый можно разделить на два подслоя.
- Входная последовательность, поступающая в энкодер, сначала проходит через слой внутреннего внимания (self-attention), помогающий энкодеру посмотреть на другие слова во входном предложении во время кодирования конкретного слова.
- Выход слоя внутреннего внимания отправляется в нейронную сеть прямого распространения (feed-forward neural network).

Декодер также содержит эти два слоя, но между ними есть слой внимания, который помогает декодеру фокусироваться на релевантных частях исходного предложения (аналогично механизму внимания в моделях seq2seq).

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/enc_dec_tr.png" width="650"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Как и в случае других алгоритмов, каждое слово входной и целевой последовательности преобразуется в вектор.

После того как слова входящего предложения преобразовались в эмбеддинги, каждый из них по отдельности проходит через слои энкодера. Одна из основных особенностей Трансформера состоит в том, что каждое слово идет по своей собственной траектории.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/encoder.png" width="700"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

## Создание модели

### Механизм внутреннего внимания

Пусть мы хотим перевести предложение: *The animal didn't cross the street because it was too tired*. Местоимение *it* может относиться к улице (*street*) или к животному (*animal*). Когда модель обрабатывает слово *it*, слой внутреннего внимания помогает понять, что *it* относится к *animal*.

По мере того как модель обрабатывает каждое слово входной последовательности, внутреннее внимание позволяет "взглянуть" на другие слова и лучше закодировать данное слово. Механизм внутреннего внимания – это метод, который Трансформер использует, чтобы смоделировать "понимание" других релевантных слов при обработке конкретного слова.

Во время кодирования *it* в энкодере №5 часть механизма внимания фокусируется на *The animal* и использует фрагмент его представления для кодирования *it*.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/self_attention.png" width="400"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

#### На примере векторов

При вычислении внутреннего внимания используются: вектор запроса $query$, вектор ключа $key$ и вектор значения $value$. Они создаются с помощью умножения эмбеддинга слова последовательности на три матрицы весов (линейных слоя) $W^Q, W^K, W^V$. Размер новых векторов – 64, размер исходных векторов – 512.

$q_i=x_iW^Q$

$k_i=x_iW^K$

$v_i=x_iW^V$

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/vector_attention.png" width="700"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Далее вычисляется коэффициент внутреннего внимания $score$ для $i$-го слова по отношению к каждому слову в предложении. Коэффициент определяет, насколько нужно сфокусироваться на других частях предложения во время кодирования слова в $i$-й позиции. Коэффициент подсчитывается с помощью скалярного произведения вектора запроса $q$ $i$-го слова и вектора ключа $k$ каждого слова.

$score_{ij}=q_i \cdot k_j$

📌 Какие векторы нужно перемножить, чтобы посчитать коэффициент внимания слова *Thinking* по отношению к слову *Machines*? По отношению к самому себе?

На следующем шаге коэффициенты делятся на квадратный корень из $d_k$ – размера векторов ключа $k$. К получившимся значениям применяется функция активации softmax, чтобы коэффициенты в сумме давали 1. Полученный софтмакс-коэффициент определяет, в какой мере каждое из слов предложения "фокусируется" на другом слове.

$softmax.score_{ij}=softmax(\frac{score_i}{\sqrt d_k})$

После этого каждый вектор значения $v$ умножается на софтмакс-коэффициент, получаем взвешенные векторы. Идея в том, что нужно сохранять без изменений значения слов, на которых мы фокусируемся, и отвести на второй план нерелевантные слова (умножив их на небольшие значения, например, 0.001). Затем  взвешенные векторы складываются. Результат (взвешенная сумма) представляет собой выход слоя внутреннего внимания для $i$-го слова.

$sum_i=\sum_{j=1}^nv_j \cdot softmax.score_{ij}$

Полученный вектор передается дальше в нейронную сеть прямого распространения.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_example.png" width="600"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

#### На примере матриц

Аналогично, матрицы запроса $Q$, ключа $K$ и значения $V$ вычисляются с помощью умножения эмбеддингов матрицы $X$ на матрицы весов (линейные слои) $W^Q, W^K, W^V$. Каждая строка в матрице $X$ соответствует слову в предложении.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/matrix_attention.png" width="450"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Последующие этапы для вычисления выхода слоя внутреннего внимания могут быть отражены в одной формуле.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_score.png" width="600"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Напишем функцию `attention` для подсчета внутреннего внимания.

$Attention(Q, K, V) = softmax\large(\frac{QK^T}{\sqrt{d_k}})V$

$Q, K, V$ — матрицы размера `batch_size, seq_length, num_features`.

Для умножения матриц по батчам используется метод `.bmm`.

Транспонирование осуществляется с помощью метода `.transpose`.

In [ ]:
import math, torch
import numpy as np
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):
    """Compute 'Scaled Dot Product Attention'
    """
    d_k = query.size(-1)
    # todo: compute the attention scores by using torch.matmul
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    # todo: compute the result as the values weighted by attention probabilities (again, using torch.matmul)
    result = torch.matmul(p_attn, value)
    return result, p_attn

In [ ]:
results, attentions = attention(
    torch.tensor([[0, 0], [0, 1], [1, 1]], dtype=torch.float),
    torch.tensor([[100, 0], [0, 100], [0, 0]], dtype=torch.float),
    torch.tensor([[1, 0], [0, 1], [0, 0]], dtype=torch.float),
)
print(results)
print(attentions)

assert np.allclose(results[0].numpy(), [1/3, 1/3])  # the first query attends to all keys equally
assert np.allclose(results[1].numpy(), [0, 1])      # the second query attends only to the second key
assert np.allclose(results[2].numpy(), [1/2, 1/2])  # the third query attends to the first and second key equally